In [ ]:
%%capture

%load_ext autoreload
%autoreload 2
%matplotlib inline
%load_ext tfl_training_sbi

In [ ]:
%presentation_style

In [ ]:
%%capture

%set_random_seed 12

In [ ]:
%load_latex_macros

<img src="_static/images/aai-institute-cover.png" alt="Snow" style="width:100%;">
<div class="md-slide title">
    <h1>Neural Density Estimation for SBI</h1>
</div>

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import sbi.analysis
import torch
import torch.nn as nn

from torch.utils.data import DataLoader
from torchvision.transforms import Compose
from tqdm.notebook import trange, tqdm

from tfl_training_sbi import mdn
from tfl_training_sbi.config import (
    default_remote_storage,
    get_config,
    root_dir,
)
from tfl_training_sbi.data_utils import (
    SIRSimulation,
    SIRStdScaler,
    load_sir_data,
)
from tfl_training_sbi.utils_sir import eval_sir_model

storage = default_remote_storage()
c = get_config(reload=True)


# Introduction to Neural Density Estimation for SBI

- We've seen Likelihood-free inference with ABC
- We've seen Conditional Density Estimation

Now, we'll combine the idea of Likelihood-free inference with Density Estimation, i.e. Simulation-Based inference (SBI)

<figure>
    <center>
    <img src="_static/images/sbi_concept_figure.png" style="width:50%"/>
    <figcaption>Fig. 1 - Schematic overview of SBI. Figure taken from Jan Boelts, 2023</figcaption>
</figure>


## Neural Density Estimation for SBI

Parameterization vs. Learning of a distribution.

Methodologies: 

- Neural Posterior Estimation (NPE) $\leftarrow$ **This notebook**
- Neural Likelihood Estimation (NLE)
- Neural Ratio Estimation (NRE)
- Neural Score Estimation (NSE)
- Flow Matching Posterior Estimation (FMPE)

...

- And even sequential variants!


## Idea of Neural Posterior Estimation

- Learn the posterior $p(\theta|x)$ of a parameter $\theta$ given data $x$ using Conditional Density Estimation
- How? Use samples from the joint $(\theta, x) \sim p(x | \theta) p(\theta)$ to learn $p(\theta|x)$
$$
p(\theta \mid x) \propto p(x \mid \theta) p(\theta) = p(\theta, x)
$$

## Sampling from the Joint Distribution

 1. Define a prior $p(\theta)$
 2. Define a simulator / generator $g(\theta)$
 3. Draw sample $\theta \sim p(\theta)$ and obtain $x = g(\theta)$, where $x\sim p(x\mid\theta)$

$$
p(\theta, x) = p(\theta) p(x \mid \theta)
$$

## Posterior decomposition

Given the joint distribution visualized in a 2-D plot: 

- the **likelihood** is a vertical slice through the joint (conditioning on $\theta$)

- the **posterior** is a horizontal slice through the joint (conditioning on $x$)

<figure>
    <center>
    <img src="_static/images/illustration_posterior.png" style="width:50%"/>
    <figcaption>Fig. 2 - Figure taken from <a href="https://mlcolab.org/simulation-based-inference-for-scientific-discovery">MLColab</a></figcaption>
</figure>

## Example: Neural Posterior Estimation with MDNs

We will learn the parameterization of Gaussian mixture model using a neural
network.

Parameters will be adapted using the maximum likelihood principle: minimizing
the negative log-likelihood of the data under the model.

We will use a Mixture Density Network (MDN) to do so, as seen before. 

Note: there are more powerful neural density estimators, e.g. Normalizing Flows, or Diffusion Models.

<figure>
    <center>
    <img src="_static/images/illustration_npe_mdn.png" style="width:50%"/>
    <figcaption>Fig. 3 - Figure taken from <a href="https://mlcolab.org/simulation-based-inference-for-scientific-discovery">MLColab</a></figcaption>
</figure>


### Example Data: SIR Dataset

For this example, the SIR dataset is used. 

**Task:** Load the data and get familiar with it. Again, a pseudo simulator is
provided. 

In [ ]:
# we provide a pre-computed dataset according to the differential equations
# defining the SIR model.
# the pseudo simulator draws samples at random
sir_theta, sir_x = load_sir_data(c.data)
# load class for z-scoring SIR data.
sir_scaler = SIRStdScaler()
simulator = SIRSimulation(
    sir_theta[:-2], sir_x[:-2], transformations=Compose([sir_scaler])
)

In [ ]:
# let's fix the last pair as observation
theta_o_raw, x_o_raw = torch.tensor(sir_theta[-2]), torch.tensor(sir_x[-2])
# z-scoring
scaled_data = sir_scaler({"theta": theta_o_raw, "x": x_o_raw})
theta_o, x_o = scaled_data["theta"], scaled_data["x"]


In [ ]:
# familiarize yourself with the data; feel free to call this cell several times
# as the observation is randomly sampled
plt.figure(figsize=(8, 6))
plt.plot(x_o_raw, "o-", alpha=1.0)
plt.title("Sample from the simulator, given infection rate and recovery rate")
plt.ylabel("Number of infections")
plt.xlabel("Sampled time steps")
plt.show()


**Task 1:** Define a `torch DataLoader` object which uses the above defined dataset with a suitable batch size. Feel free to experiment with `shuffle` and `drop_last` options. Why and how would you use them?

In [ ]:
# %load -r 8-11 ./solutions/solutions_nb_03.py

**Task 2:** We're using a mixture of Gaussians to approximate the posterior. The here used `MDN` module allows to provide a `hidden_net` for feature extraction. Define a fully connected feature extractor and initialize a Mixture Density Network as density estimator. Feel free to use the below structure.

Furthermore, think about the following parameters. What did you choose?

- `features`: number of variable in the problem
- `num_components`: number of Gaussians in the mixture 
- `hidden_features`: output dim. of the linear layer from the feature extractor 


```python
<YOUR-FEAT-EXTRACTOR> = nn.Sequential(
    ...,
)

mixture_density_net = mdn.MultivariateGaussianMDN(
    features=<TODO>,
    hidden_net=<YOUR-FEAT-EXTRACTOR>,
    num_components=<TODO>,
    hidden_features=<TODO>,
)

```

In [ ]:
# %load -r 14-30 ./solutions/solutions_nb_03.py

In [ ]:
loss_hist = []
num_epochs = 30

# fit MDN using negative log-likelihood loss
opt = torch.optim.Adam(mixture_density_net.parameters(), lr=0.001)
for e in trange(num_epochs):
    for batch in sir_dataloader:
        theta_batch, x_batch = batch["theta"], batch["x"]

        opt.zero_grad()

        # get the mixture components
        (
            weights_of_each_gaussian,
            means,
            variances,
        ) = mixture_density_net.get_mixture_components(x_batch)

        # compute Likelihood of sample under current parameterization
        log_probs = mdn.mog_log_prob(
            theta_batch, weights_of_each_gaussian, means, variances
        )

        # compute the negative log-likelihood loss
        loss = -log_probs.sum()
        loss.backward()

        # track  the loss per batch
        loss_hist.append(loss.item())

        # update the parameters using gradient descent
        opt.step()


In [ ]:
plt.figure(figsize=(12, 4))
plt.plot(loss_hist)
plt.xlabel("Step")
plt.ylabel("Loss")
plt.show()


## Great, you've run through your first SBI workflow! 🎉

 1. Define a prior $p(\theta)$
 2. Define a simulator / generator $g(\theta)$
 3. Draw sample $\theta \sim p(\theta)$ and obtain $x = g(\theta)$, where $x\sim p(x\mid\theta)$
 4. Train density estimator $q_{\phi}(\theta \mid x) \approx p(\theta \mid x)$ using simulated data.


## Inspecting the Approximated Posterior Distribution

We have trained a MDN to approximate the posterior distribution over SIR parameters, $q_{\phi}(\theta \mid
x_o)$. 

Below, we will inspect the learned posterior distribution in two steps:

**Step 1**. Drawing samples and visualizing the two-dimensional posterior distribution

**Step 2**. Simulating data with samples from the posterior and compare to the original observation $x_o$.


**Task 3:** Use the trained posterior approximator to sample from the conditional distribution, given the test observation. 

Therefore, obtain the parameters for the Gaussian mixture using the `.get_mixture_components(..)` method of the density estimator object. Then, use the `mog_sample(..)` function of the `mdn` module.

In order to stick with following cells, please store the obtained samples in a tensor called `samples`.

In [ ]:
# %load -r 33-45 ./solutions/solutions_nb_03.py

In [ ]:
# rescale to original parameter ranges.
samples_unscaled = sir_scaler.rescale(
    {"theta": samples, "x": torch.ones_like(samples)}
)["theta"]

# eval. the SIR model for a selected number of samples
sample_obs = []
subsampling = 16  # we subsample to get 10 time points.
for i in trange(1_000):
    x = eval_sir_model(theta=samples_unscaled[i])
    sample_obs.append(x[::subsampling])  

# stack the collected samples into a big array of shape (N, 160, 3)
sample_obs = np.stack(sample_obs, axis=0)

### Step 1: Visualizing the Resulting Posterior Approximation

Plot the marginal distributions per parameter and the joint distribution of the
parameters.


In [ ]:
sbi.analysis.pairplot(
    [sir_theta[:10000], samples_unscaled[:10000]], points=theta_o_raw, points_offdiag={"markersize": 10}, 
    points_colors="r", limits=[[0.0, 2.5], [0.05, 0.25]], labels=["infection rate", "recovery rate"], 
    offdiag="scatter", legend_kwargs=dict(labels=["prior", "posterior", "theta_o"]), legend=True, 
    hist_diag=dict(density=True, bins="auto"),
    figsize=(6, 6)
);

### Step 2: Perform a **posterior predictive check**

Simulate with parameters sampled from the posterior and compare to observed data $x_o$. 

In [ ]:
# eval the SIR model for the true parameters to get a full scale observation
x_o_full_scale = eval_sir_model(theta=theta_o_raw.numpy())


plt.figure(figsize=(12, 4))
plt.plot(sample_obs[:, :, 1].T, "y--", alpha=.15)
plt.plot([], [], "y--", label="Simulated from posterior")
plt.plot(x_o_full_scale[::subsampling, 1], "-", alpha=1.0, label="$x_o$")

plt.xlabel("time $t$")
plt.ylabel("Proportion of infected people (I)")
plt.legend()

plt.show()

## What if `x` is high-dimensional (image, time series, ...)

- Subsampling
- Summary Statistics
- New: we can learn feature embeddings from data using embedding nets!

<figure>
    <center>
    <img src="_static/images/illustration_mdn_feat_extraction.png" style="width:50%"/>
    <figcaption>Fig. 4 - Pre-pending a powerful feature extractor might be necessary in order to obtain meaningful features from time series data. Figure taken from <a href="https://mlcolab.org/simulation-based-inference-for-scientific-discovery">MLColab</a></figcaption>
</figure>

## Sequential SBI


### NPE is fully amortized
- NPE is trained on a large range of $x$ (from the prior): 
    - we can get $p(\theta \mid x_o)$ for many different $x_o$
- What if we are interested in only one particular $x_o$? 


### Focusing inference on one $x_o$

- Sample inefficiency is a problem of SBI, esp. for costly simulators
- Many samples from the prior yield low likelihood / are not informative about $x_o$
- **Idea:** re-condition the prior sequentially on $x_o$ to narrow down the "search space" --> Sequential Estimation

## Example: Sequential Posterior Estimation (SNPE)

1. Train a neural density estimator $\hat{p}_{\phi}^{(i)}(\theta \mid
   \mathbf{x})$ based on $\theta \sim p^{(i-1)}(\theta)$
2. Substitute the prior $p^{(i-1)}(\theta)$ by $\hat{p}_{\phi}^{(i)}(\theta)$
3. Train $\hat{p}_{\phi}^{(i+1)}(\theta \mid \mathbf{x})$ based on $\hat{p}_{\phi}^{(i)}(\theta)$
4. Iterate until convergence

<figure>
    <center>
    <img src="_static/images/illustration_snpe.png" style="width:35%"/>
    <figcaption>Fig. 5 - Illustrating the iterative refinement of the posterior approximation with multi-round inference. Figure obtained JM Lueckmann.</figcaption>
</figure>

## The Problem with SNPE

 - We want to obtain $\hat{p}(\theta \mid \mathbf{x}) \approx p(\theta \mid \mathbf{x}) = \frac{p(x \mid \theta) p(\theta)}{p(x)}$.
 - We use a proposal prior instead of the real prior, e.g., $\tilde{p}(\theta) = q_{\phi}(\theta \mid x)$
 - When we change the prior to $\tilde{p}(\theta)$, we are also changing the posterior.
$$
    \tilde{\hat{p}}(\theta \mid \mathbf{x}) \approx p(\mathbf{x} \mid \theta) \tilde{p}(\theta)
$$
 - Consequence: We need to correct for the change in the prior, to get the actual posterior. 

## Popular Methods for SNPE

All methods introduce a different method to correct the posterior approximation. 

- SNPE-A (Papamakarios, Murray, 2016, _"Fast $\epsilon$-free Bayesian inference..."_)
- SNPE-B (Lueckmann, Goncalves, et al. 2017, _"Flexible statistical inference..."_)
- SNPE-C (APT) (Greenberg, Nonnenmacher, Macke, 2019, _"Automatic posterior transformation..."_)

**Note**: Without the sequential approach, all SNPE methods are the same and train by
maximizing the log-Likelihood.

More recent NPE methods: 
- FMPE: flow matching neural posterior estimation - density estimation using a continuous normalizing flow (Wildberger, Dax et al., 2023, "Flow matching for scalable SBI")
- NPSE: neural posterior score estimation - using score-matching diffusion models to approximate the posterior (Sharrock et al. 2022, Geffner et al. 2023)

## Summary

#### Neural Posterior Estimation works by

  1. Sampling from the prior $\theta \sim p(\theta)$
  2. Drawing samples from the joint by forward propagation $\mathbf{x} \sim p(\mathbf{x}|\theta)$
  3. Train a conditional density estimator $\hat{p}_{\phi}(\theta|\mathbf{x})$
on the samples $\{ (\theta, \mathbf{x}) \}^N_{i=1}$
  4. Evaluate the conditional density estimator at certain observations $\mathbf{x}_o$

#### Benefits 

 - NPE is **amortized** s.t. no re-training is required for new observations;
   contrary to ABC
 - (Embedding) Networks allow to cope with high-dimensional data

#### Sequential NPE

 - Enhances **sample efficiency**
 - Requires adaptations to correct the diverging posterior

## Learning goals

1) How is neural density estimation used for SBI?
2) What is Neural Posterior Estimation?
3) What is a posterior predictive check? 
4) What does amortization mean in the context of SBI?
5) What is the motivation for sequential SBI?

<img src="_static/images/aai-institute-cover.png" alt="Snow" style="width:100%;">
<div class="md-slide title">
    <h1>Thank you for your attention!</h1>
</div>